## **Preprocessing**

**Mount Google Drive to Local Machine**

First the directory containing the data must be mounted to the Colab virtual machine


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Once drive has been mounted images can be accessed by the virtual machine for manipulation in the preprocessing phase

In [ ]:
training_data_path = '/content/drive/My Drive/AD Project/Alzheimers/train'
test_data_path = '/content/drive/My Drive/AD Project/Alzheimers/test'

**Use SimpleITK to Correct for Bias Field Correction**

In this stage we will commence the preprocessing of the images by applying an N4 field bias correction



In [ ]:
!pip install simpleitk

In [ ]:
import SimpleITK as sitk
import sys
import os
import matplotlib.pyplot as plt
import cv2

In [ ]:
%cd '/content/drive/My Drive/AD Project/Alzheimers/test/Final AD JPEG' 

In [ ]:
import SimpleITK as sitk


fileName = "xxx.jpg"

image = sitk.ReadImage(fileName, imageIO='JPEGImageIO')

print(image.GetPixelID())
print(sitk.GetPixelIDValueAsString(image.GetPixelID()))


In [ ]:
!ls '/content/drive/My Drive/AD Project/Alzheimers/test/Final AD JPEG' 

In [ ]:
import sys
import os
import matplotlib.pyplot as plt

#show image before preprocessing in grayscale

plt.imshow(sitk.GetArrayViewFromImage(image), cmap='gray')


Below is iterating through the root directory and then applying the filter to each image in the set - this will preserve the structure of the files while ensuring that the SimpleITK n4 Bias Field Correction Algorithm is applied





In [ ]:
#Preprocessing bias field correction test

testImagePath = '/content/drive/My Drive/AD Project/Alzheimers/test/Final AD JPEG/xxx.jpg' 

testImage = sitk.ReadImage(testImagePath, imageIO='JPEGImageIO')

maskImage = sitk.OtsuThreshold(testImage, 0, 1, 200)

testImage = sitk.Cast(testImage, sitk.sitkFloat32)

biasCorrection = sitk.N4BiasFieldCorrectionImageFilter()

appliedCorrection = biasCorrection.Execute(testImage, maskImage)

correctedOutput = sitk.Cast(appliedCorrection,sitk.sitkVectorUInt8)

outputPath = '/content/drive/My Drive/AD Project/Preprocessing Test/test.jpg'

sitk.WriteImage(correctedOutput, outputPath)


In [ ]:
outputPath = '/content/drive/My Drive/AD Project/Preprocessing Test/test.jpg'

correctedImageTest = sitk.ReadImage(outputPath, imageIO='JPEGImageIO')

plt.imshow(sitk.GetArrayViewFromImage(correctedImageTest), cmap='gray')

In [ ]:
rootDirectory = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed/'

#iterate through images applying the N4 Bias Field Correction to preprocessed images as shown above

for subdirs, dirs, files in os.walk(rootDirectory):
  for file in files:
    if file.endswith('.jpg'):
      
      imageToProcess = subdirs + '/' + file

      print(imageToProcess)

      testImage = sitk.ReadImage(imageToProcess, imageIO='JPEGImageIO')

      maskImage = sitk.OtsuThreshold(testImage, 0, 1, 200)

      testImage = sitk.Cast(testImage, sitk.sitkFloat32)

      biasCorrection = sitk.N4BiasFieldCorrectionImageFilter()

      appliedCorrection = biasCorrection.Execute(testImage, maskImage)

      correctedOutput = sitk.Cast(appliedCorrection,sitk.sitkVectorUInt8)

      outputPath = imageToProcess

      sitk.WriteImage(correctedOutput, outputPath)

print("N4 Bias Field Correction Applied to the files in Preprocessed folder")


Comparison of 2 images to ensure bias field correction has been successfully applied

In [ ]:
#paths for images to compare in unprocessed folder vs preprocessed folder

nonCorrectedImage1Path = '/content/drive/My Drive/AD Project/Alzheimers-Data/train/Final CN JPEG/xxx.jpg'
nonCorrectedImage2Path = '/content/drive/My Drive/AD Project/Alzheimers-Data/train/Final CN JPEG/xxx.jpg'
correctedImage1Path = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed/train/Final CN JPEG/xxx.jpg'
correctedImage2Path = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed/train/Final CN JPEG/xxx.jpg'

nonCorrectedImage1 = sitk.ReadImage(nonCorrectedImage1Path, imageIO='JPEGImageIO')
nonCorrectedImage2 = sitk.ReadImage(nonCorrectedImage2Path, imageIO='JPEGImageIO')
correctedImage1 = sitk.ReadImage(correctedImage1Path, imageIO='JPEGImageIO')
correctedImage2 = sitk.ReadImage(correctedImage2Path, imageIO='JPEGImageIO')

#configure figures

figure = plt.figure(figsize=(10,10))

rows = 2
columns = 2

figure.add_subplot(rows,columns, 1)

#add subplots to the figure for comparison of N4 Bias Filtered Images with titles

plt.imshow(sitk.GetArrayViewFromImage(nonCorrectedImage1), cmap='gray')
plt.title("No N4 Bias Field Correction 1")

figure.add_subplot(rows,columns, 2)

plt.imshow(sitk.GetArrayViewFromImage(nonCorrectedImage2), cmap='gray')
plt.title("No N4 Bias Field Correction 2")

figure.add_subplot(rows,columns, 3)

plt.imshow(sitk.GetArrayViewFromImage(correctedImage1), cmap='gray')
plt.title("N4 Bias Field Corrected 1")

figure.add_subplot(rows,columns, 4)

plt.imshow(sitk.GetArrayViewFromImage(correctedImage2), cmap='gray')
plt.title("N4 Bias Field Corrected 2")


**Intensity Normalisation**

This section takes the impges with the N4 Bias Field Correction and then applies the MR normalisation method pioneered by Nyul & Udupa (2000) 

In [ ]:
#Preprocessing intensity normalisation test

testImagePath = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed/train/Final AD JPEG/xxx.jpg' 

referenceImagePath = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed/train/Final AD JPEG/xxx.jpg'

testImage = sitk.ReadImage(testImagePath, imageIO='JPEGImageIO')

referenceImage = sitk.ReadImage(referenceImagePath, imageIO='JPEGImageIO')

testImageCast = sitk.Cast(testImage, sitk.sitkFloat32)

refImageCast = sitk.Cast(referenceImage, sitk.sitkFloat32)

normalisationFilter = sitk.HistogramMatchingImageFilter()

appliedNormalisation = normalisationFilter.Execute(testImageCast, refImageCast)

correctedOutput = sitk.Cast(appliedNormalisation,sitk.sitkVectorUInt8)

outputPath = '/content/drive/My Drive/AD Project/Preprocessing Test/normalisationtest.jpg'

sitk.WriteImage(correctedOutput, outputPath)

In [ ]:
#Display image before intensity normalisation

preNormTestPath = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed/train/Final AD JPEG/xxx.jpg'

preCorrectedImageTest = sitk.ReadImage(preNormTestPath, imageIO='JPEGImageIO')

plt.imshow(sitk.GetArrayViewFromImage(preCorrectedImageTest), cmap='gray')


In [ ]:
#Display image after intensity normalisation 

normTestPath = '/content/drive/My Drive/AD Project/Preprocessing Test/normalisationtest.jpg'

correctedImageTest = sitk.ReadImage(normTestPath, imageIO='JPEGImageIO')

plt.imshow(sitk.GetArrayViewFromImage(correctedImageTest), cmap='gray')



In [ ]:
rootDirectory = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed-N4/'

referenceImagePath = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed/train/Final AD JPEG/xxx.jpg'

#iterate through images applying MR intensity normalisation to preprocessed images as shown above

for subdirs, dirs, files in os.walk(rootDirectory):
  for file in files:
    if file.endswith('.jpg'):
      
      imageToProcess = subdirs + '/' + file

      print(imageToProcess)

      testImage = sitk.ReadImage(imageToProcess, imageIO='JPEGImageIO')

      referenceImage = sitk.ReadImage(referenceImagePath, imageIO='JPEGImageIO')

      testImageCast = sitk.Cast(testImage, sitk.sitkFloat32)

      refImageCast = sitk.Cast(referenceImage, sitk.sitkFloat32)

      normalisationFilter = sitk.HistogramMatchingImageFilter()

      appliedNormalisation = normalisationFilter.Execute(testImageCast, refImageCast)

      correctedOutput = sitk.Cast(appliedNormalisation,sitk.sitkVectorUInt8)

      outputPath = imageToProcess

      sitk.WriteImage(correctedOutput, outputPath)

print("Nyul & Udupa (2000) MR intensity normalisation filter applied to to the files in preprocessed-N4 folder")

At this point the N4 Bias Field Correction Filter has been applied along with the intensity normalisation filter using a reference image among the images - this is primarily so that subtleties can be detected between images


**Resampling and Resizing**

At this point the images must be resampled and resized before the final image registration stage to map images onto the same pixel base

In [ ]:
#load images from different paths using the sitk image reader - test image is 336x336 pixels, reference image is 256x256 pixels

testImagePath = '/content/drive/My Drive/AD Project/Alzheimers-Data/train/Final EMCI JPEG/xxx.jpg' 

referenceImagePath = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed/train/Final AD JPEG/xxx.jpg'

testImage = sitk.ReadImage(testImagePath, imageIO='JPEGImageIO')

refImage = sitk.ReadImage(referenceImagePath, imageIO='JPEGImageIO')

#obtain information regarding the reference image ready for application to test image

referenceSpacing = refImage.GetSpacing()

referenceSize = refImage.GetSize()

#intitialis the resampler filter and pass thorugh the required parameters

resamplerFilter = sitk.ResampleImageFilter()

resamplerFilter.SetReferenceImage(refImage)

#resamplerFilter.SetSize(referenceSize)
#resamplerFilter.SetOutputSpacing(referenceSpacing)
#resamplerFilter.SetInterpolator(sitk.sitkBSpline)

#apply the filter to the test image

resampledTest = resamplerFilter.Execute(testImage)

outputPath = '/content/drive/My Drive/AD Project/Preprocessing Test/resamplingtest2.jpg'

sitk.WriteImage(resampledTest, outputPath)



**Image Registration**

A non-rigid co-registration process will help to align the between subject images together, meaning that the neural network will be able to more efficiently use the pixel information in the image as they will be comparable

In [ ]:
#Preprocessing registration test

testImagePath = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed/train/Final AD JPEG/.jpg' 

referenceImagePath = '/content/drive/My Drive/AD Project/Alzheimers-Data-Preprocessed/train/Final AD JPEG/xxx.jpg'

movingImage = sitk.ReadImage(testImagePath, imageIO='JPEGImageIO')

fixedImage = sitk.ReadImage(referenceImagePath, imageIO='JPEGImageIO')

movingImageCast = sitk.Cast(movingImage, sitk.sitkFloat32)

fixedImageCast = sitk.Cast(fixedImage, sitk.sitkFloat32)

#Set the registration method - B Spline as non-ridgid and supported by simple itk

registrationMethod = sitk.ImageRegistrationMethod()

#Set the comparison metric by which the registraiton is measured

registrationMethod.SetMetricAsMeanSquares()

#Set the interpolation method - this is key to the 

registrationMethod.SetInterpolator(sitk.sitkLinear)

registeredImage = registrationMethod.Execute(fixedImageCast, movingImageCast)

registeredImage = sitk.Cast(appliedNormalisation,sitk.sitkVectorUInt8)

outputPath = '/content/drive/My Drive/AD Project/Preprocessing Test/registeringtest.jpg'

sitk.WriteImage(registeredImage, outputPath)